# To access the Mongo_db follow instructions:

To dump your mongo_db call this command on your terminal

 - mongodump --db database_name --collection collection_name (It will create a folder called dump in the folder active directory)

To import mongo_db file to mongodb you can use the following command on your terminal:

 - mongorestore --db database_name path_to_bson_file

In [1]:
# Database dependencies.
import pymongo
 
# Api dependencies
import json
import requests 

#Json dependencies
from pprint import pprint

#API key
from DriversConfig.config import API_KEY #Save API key as API_KEY in a folder called DriversConfig

#DataFrame dependencies
import pandas as pd 

#Webscrapping 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

#Random crap
import time

#Settings for headless mode.
options = webdriver.ChromeOptions()
options.add_argument('headless')

#path to the driver and load the options.
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Group Project2 - ProjectEnergy/Sebastian/DriversConfig/chromedriver",chrome_options = options) #Dowload Selenium chromedriver and save the driver in the folder in the repo and update Absolute path.


#Mongo to Json 
from bson import json_util 
import json 


In [2]:
# Database connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection --- creates database.
db = client.USEnergy2

In [3]:
#Lower 48 list.
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Prices 

- Data is very structured, SQL can be a good idea here

In [4]:
#Define collection for prices
collection = db.Prices

#API key from config file.
api_key = API_KEY

#Empty dictionary to store prices
price_df = pd.DataFrame(columns = states)

#for state in states.
for state in states:

    #query each state. ALL = average combined rates, A = Annual. --> we can get more specific and get Residential, bussiness, etc. 
    priceQuery = f"ELEC.PRICE.{state}-ALL.A"

    #Api url.
    url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

    #Api reqeust. 
    statePriceRequest = requests.get(url)

    #Pulls list of list of with the information of prices and yaer.
    yearPricesList = statePriceRequest.json()['series'][0]['data']
    
    #Parse through the list and build DataFrame.
    for priceYear in yearPricesList:

        price_df.loc[priceYear[0],state] = priceYear[1]

#Converts the dataframe to a dictionary. 
for state in price_df.to_dict().keys():

    post = {
        "State" : state,
        "Energy Price" : price_df.to_dict()[state]
    }
    collection.insert_one(post)




# Energy Generation

- Data is very unstructured, we might have to use NoSQL (Mongo_dB)


<img src="images/2019 US Energy Distribution.png">

In [7]:
#Define collection for Energy Generation.
collection = db.EnegryGeneration

#list of the US Energy Sources by EIA.
energySources = {"Natural Gas" : "NG",
                "Coal" : "COW",
                "Petroleum Liquids" : "PEL", 
                "Petroleum Coke" : "PC", 
                "Other Gasses": "OOG",
                "Nuclear" : "NUC",
                "Hydroelectric" : 'HYC',
                "Wind" : "WND",
                'Wood and Wood Derived Fuels' : 'WWW',
                'Other Biomass' : 'WAS',
                'Geothermal' : 'GEO',
                'All Other Renewables' : "AOR",
                "All Solar": 'TSN',
                'Other' : 'OTH',
                'Utility Scale Thermal' : 'STH',
                'Hydroelectic Pumped Storage' : 'HPS'
                }

#For state in states
for state in states:

    #Dictionary to build.
    energySources_dict = {}
    
    try:
        #For every statem loop each energy source.
        for energySourceKey, energySourceValue in zip(energySources.keys(), energySources.values()):
            
            #builds api query.
            priceQuery = f"ELEC.GEN.{energySourceValue}-{state}-99.A"
            
            #API url
            url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

            #Api reqeust. 
            energySourceRequest = requests.get(url, allow_redirects = False) 

            #Extracts the year-energy liss
            energySourceList = energySourceRequest.json()['series'][0]['data']

            #builds the state keys.
            if state not in energySources_dict.keys():
                energySources_dict[state] = {}

            #builds the energy source keys by year and energy production by source.
            if energySourceKey not in energySources_dict[state].keys():
                energySources_dict[state][energySourceKey] = {}
                energySources_dict[state][energySourceKey]['Year'] = []
                energySources_dict[state][energySourceKey]['K Megawatthours'] = []
                
            #builds the list for the years
            [energySources_dict[state][energySourceKey]['Year'].append(i[0]) for i in energySourceList]

            #Builds list for the values of each year.
            [energySources_dict[state][energySourceKey]['K Megawatthours'].append(i[1]) for i in energySourceList]
    
    #if energy surce is not found.
    except KeyError as K:
        print(f'{K}. {energySourceKey} not found in {state}.')

        time.sleep(5)

    post = {
        'State' : state,
        "Data" : energySources_dict
    }
    collection.insert_one(post)

print('Done!')

'series'. Petroleum Coke not found in AL.
'series'. Petroleum Coke not found in AK.
'series'. Petroleum Coke not found in AZ.
'series'. Other Gasses not found in AR.
'series'. Petroleum Coke not found in CO.
'series'. Petroleum Coke not found in CT.
'series'. Coal not found in DC.
'series'. Nuclear not found in DE.
'series'. Geothermal not found in FL.
'series'. Other Gasses not found in GA.
'series'. Natural Gas not found in HI.
'series'. Petroleum Coke not found in ID.
'series'. Geothermal not found in IL.
'series'. Nuclear not found in IN.
'series'. Other Gasses not found in IA.
'series'. Other Gasses not found in KS.
'series'. Nuclear not found in KY.
'series'. Wind not found in LA.
'series'. Petroleum Coke not found in ME.
'series'. Petroleum Coke not found in MD.
'series'. Petroleum Coke not found in MA.
'series'. Geothermal not found in MI.
'series'. Geothermal not found in MN.
'series'. Petroleum Coke not found in MS.
'series'. Geothermal not found in MO.
'series'. Nuclear not 

# Energy Consumption by State

<img src="images/Energy Classification.png">


In [4]:
#Define collection for Energy Generation.
collection = db.EndUseCosumption

# Empty dataframe to store the data 
EndUseCosumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(3)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-electricity'] 

    for parseClass in parseClasses:

        yummySoup = soup.find_all("li", class_={parseClass})[0].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        EndUseCosumption_df.loc[typeEnergy,state] = marketShare

#Converts the dataframe to a dictionary. 
for state in EndUseCosumption_df.keys():

    post = {
        "State" : state,
        "End Use Consumption" : EndUseCosumption_df.to_dict()[state]
    }
    collection.insert_one(post)

In [5]:
#Define collection for Energy Generation.
collection = db.ElectricPowerConsumption

#Empty dataframe to store the data 
ElectricPowerConsumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(4)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-nuclear'] 

    for parseClass in parseClasses:

        #From the soup, the electricity generation is in the index 1.
        indexNum = 1

        #Nuclear energy is only for electricity generation. It only has 1 index = 0
        if parseClass == 'info-nuclear' :
            indexNum = 0

        yummySoup = soup.find_all("li", class_={parseClass})[indexNum].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        ElectricPowerConsumption_df.loc[typeEnergy,state] = marketShare

#Converts the dataframe to a dictionary. 
for state in ElectricPowerConsumption_df.keys():

    post = {
        "State" : state,
        "Electric Power Consumption" : ElectricPowerConsumption_df.to_dict()[state]
    }
    collection.insert_one(post)
